In [1]:
import gzip
import pickle
import numpy as np
import scipy
import time

In [29]:
def power_method(A, iter_num=1000, eps=1e-6):
    x = np.random.rand(A.shape[1])
    lam_prev = float('inf')
    for i in range(iter_num):
        x = A @ x
        x = x / np.linalg.norm(x)
        lam = (x.T @ A @ x) / (x.T @ x)
        if abs(lam - lam_prev) < eps : break
        lam_prev = lam
    return x, lam


def KED(adj_list, seed_idx, del_edge_num, use_npy=False):
    time_alg = -time.time()
    # convert adj list to adj matrix & make edge list
    n = len(adj_list)
    adj_mat = [[0]*n for _ in range(n)]
    edge = []
    for u in range(n):
        for v,p in adj_list[u]:
            adj_mat[u][v] = p
            edge.append([u,v,p])
    adj_mat = np.array(adj_mat)

    # calculate eigenvalue, eigenvector 
    if use_npy==False:
        Evec_r, Eval_r = power_method(adj_mat)
        Evec_l, Eval_l = power_method(adj_mat.T)
    if use_npy==True:
        eigenValues, eigenVectors = np.linalg.eig(adj_mat)
        idx = eigenValues.argsort()[::-1]   
        eigenValues = eigenValues[idx]
        eigenVectors = eigenVectors[:,idx]
        Evec_r = eigenVectors[0]
        Eval_r = eigenValues[0]
        eigenValues, eigenVectors = np.linalg.eig(adj_mat.T)
        idx = eigenValues.argsort()[::-1]   
        eigenValues = eigenValues[idx]
        eigenVectors = eigenVectors[:,idx]
        Evec_l = eigenVectors[0]
        Eval_l = eigenValues[0]

    if min(Evec_r)<0: Evec_r=-Evec_r
    if min(Evec_l)<0: Evec_l=-Evec_l

    # sort about score
    edge.sort(key=lambda x: Evec_l[x[0]]*Evec_r[x[1]], reverse=True)

    time_alg += time.time()
    return edge[:del_edge_num], time_alg